# Repartition

In [1]:
import dask
import distributed
import cudf
import os

In [2]:
print(f'Dask: {dask.__file__}')
print(f'Distributed: {distributed.__file__}')
print(f'cuDF: {cudf.__file__}')

Dask: /home/nfs/ashwint/miniconda3/envs/cudf_dev_92/lib/python3.7/site-packages/dask/__init__.py
Distributed: /home/nfs/ashwint/miniconda3/envs/cudf_dev_92/lib/python3.7/site-packages/distributed/__init__.py
cuDF: /home/nfs/ashwint/miniconda3/envs/cudf_dev_92/lib/python3.7/site-packages/cudf-0.10.0a0+800.g6407f50-py3.7-linux-x86_64.egg/cudf/__init__.py


In [3]:
base_env = {
    "UCX_RNDV_SCHEME": "put_zcopy",
    "UCX_MEMTYPE_CACHE": "n",
    "UCX_TLS": "rc,cuda_copy,cuda_ipc",
    "CUDA_VISIBLE_DEVICES": "0,1",
}
os.environ.update(base_env)

In [4]:
from dask.distributed import Client, wait
from dask_cuda import DGX

cluster = DGX(CUDA_VISIBLE_DEVICES=[0,1], 
              dashboard_address='10.33.227.165:8789')
client = Client(cluster)
# client = Client('ucx://10.33.225.165:42841')
client

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at: ucx://10.33.225.165:54590
distributed.scheduler - INFO -   dashboard at:        10.33.225.165:8789
distributed.nanny - INFO -         Start Nanny at: 'ucx://10.33.225.165:36984'
distributed.nanny - INFO -         Start Nanny at: 'ucx://10.33.225.165:59366'
distributed.scheduler - INFO - Register ucx://10.33.225.165:54571
distributed.scheduler - INFO - Starting worker compute stream, ucx://10.33.225.165:54571
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register ucx://10.33.225.165:35915
distributed.scheduler - INFO - Starting worker compute stream, ucx://10.33.225.165:35915
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Receive client connection: Client-33ec9718-c914-11e9-bf60-d8c49764f624
distributed.core - INFO - Starting established connection


Client Scheduler: ucx://10.33.225.165:54590 Dashboard: http://10.33.225.165:8789/status,Cluster Workers: 2 Cores: 2 Memory: 270.53 GB


In [5]:
import dask.array as da


In [ ]:

for i in range(20):
    n_rows = 200
    npartitions = 10
    new_npartitions = 100
    x = da.random.random(n_rows, chunks=n_rows // npartitions).to_dask_dataframe(columns='x')
    res_x = x.map_partitions(cudf.from_pandas)
    res_x = res_x.persist()
    res = res_x.repartition(npartitions=new_npartitions)
    y = res.compute()

In [ ]:
res.compute()